# BGG Scraper

This script is optimized to collect information from BGG web pages using their XMLAPI2 API.

All games are displayed on a search page provided by the site. We use that as a starting point for the scraper. With games ID (witch are unique) we are able to collect even more information about them, specialy the Statistics data.

The API alow us to query for more than 1 game, making the extraction incredibly more efficient, restricted only by the size of the URL itself. There for, we have to break the URLs into smaller groups. In this case, 500 games per request is enought to get the date without overloading the downloaded string.

In [1]:
"""Boardgame class"""

class BoardGame(object):
    """Object containing information about a boardgame"""
    
    def __init__(self, data):
        self._data = data
    
    def __len__(self):
        return self._data_len
    
    def __repr__(self):
        return("Boardgame(" + self.name + ")")
    
    def data(self):
        """Internal data dictionary"""
        return self._data
    
    @property
    def item(self):
        """Internal dictionary of 'item'"""
        return self._data #['items']['item']
        
    @property
    def statistics(self):
        """Internal dictionary of 'statistics'"""
        return self.item['statistics']
    
    @property
    def name(self):
        """object name"""
        try:
            return str(self.item['name']['@value'])
        except:
            return str(self.item['name'][0]['@value'])

            
    @property
    def id(self):
        """BGG ID"""
        return self.item['@id']
        
    @property
    def description(self):
        """Game description"""
        return self.item['description']
        
    @property
    def year_published(self):
        """Year of publication"""
        try:
            temp = int(self.item['yearpublished']['@value'])
        except:
            temp = 0
        
        return temp
        
    @property
    def min_players(self):
        """Minimum number of players, according to publisher"""
        try:
            temp = int(self.item['minplayers']['@value'])
        except:
            temp = 0
        
        return temp
        
    @property
    def max_players(self):
        """Maximum number of players, according to publisher"""
        try:
            temp = int(self.item['maxplayers']['@value'])
        except:
            temp = 0
        
        return temp

    @property
    def suggested_players(self):
        """BGG users suggested best player count"""
        poll = self.item['poll']
        poll_names = [ii['@name'] for ii in poll]
        poll = poll[poll_names.index('suggested_numplayers')]['results']
        
        best_player_cnt = {}
        def _get_best_num_players(index):
            numplay = poll[index]['@numplayers']
            best_votes = poll[index]['result'][0]['@numvotes']
            best_player_cnt[numplay] = int(best_votes)
        
        for x in range(len(poll)):
            _get_best_num_players(x) 
        max_votes = max(list(best_player_cnt.values()))
        
        suggested_players = [count for count, votes in best_player_cnt.items() if votes == max_votes][0]
                
        return suggested_players
    
    @property
    def min_age(self):
        """Publisher defined minimum age"""
        try:
            temp = int(self.item['minage']['@value'])
        except:
            temp = 0
        
        return temp
    
    @property
    def suggested_player_age(self):
        """BGG users suggested minimum age"""
        poll = self.item['poll']
        poll_names = [ii['@name'] for ii in poll]
        poll = poll[poll_names.index('suggested_playerage')]['results']['result']
        
        best_player_age = {}
        def _get_best_age_players(index):
            age = poll[index]['@value']
            num_votes = poll[index]['@numvotes']
            best_player_age[age] = int(num_votes)
        
        for x in range(len(poll)):
            _get_best_age_players(x)
        
        max_votes = max(list(best_player_age.values()))
        
        suggested_age = [count for count, votes in best_player_age.items() if votes == max_votes][0]
        
        return int(suggested_age)
    
    @property
    def language_dependence(self):
        """BGG users suggested language dependence"""
        poll = self.item['poll']
        poll_names = [ii['@name'] for ii in poll]
        poll = poll[poll_names.index('language_dependence')]['results']['result']
        
        voted_language_dependence = {}
        def _get_language_requirements(index):
            language_level = poll[index]['@value']
            num_votes = poll[index]['@numvotes']
            voted_language_dependence[language_level] = int(num_votes)
        
        for x in range(len(poll)):
            _get_language_requirements(x)
            
        max_votes = max(list(voted_language_dependence.values()))
        language_requirement = [count for count, votes in voted_language_dependence.items() if votes == max_votes][0]
        
        return language_requirement
        
    @property
    def playing_time(self):
        """BGG suggested playing time"""
        return int(self.item['playingtime']['@value'])
        
    @property
    def min_play_time(self):
        """Publisher defined minimum play time"""
        return int(self.item['minplaytime']['@value'])
        
    @property
    def max_play_time(self):
        """Publisher defined maximum play time"""
        return int(self.item['maxplaytime']['@value'])
        
    @property
    def categories(self):
        """List of game categories (i.e. theme)"""
        link = self.item['link']
        link_names = [ii['@type'] for ii in link]
        names_index = [i for i, x in enumerate(link_names) if x == 'boardgamecategory']
        categories = [link[ii]['@value'] for ii in names_index]
        
        return categories
        
    @property
    def mechanics(self):
        """List of game categories (i.e. tile-laying, set collection)"""
        link = self.item['link']
        try:
            link_names = [ii['@type'] for ii in link]
            names_index = [i for i, x in enumerate(link_names) if x == 'boardgamemechanic']
            mechanics = [link[ii]['@value'] for ii in names_index]
        except:
            link_names = 'NEDD A FIX'
        return mechanics
        
    @property
    def game_family(self):
        """List of game families (i.e. Kickstarter, Made in Canda)"""
        link = self.item['link']
        link_names = [ii['@type'] for ii in link]
        names_index = [i for i, x in enumerate(link_names) if x == 'boardgamefamily']
        family = [link[ii]['@value'] for ii in names_index]
        
        return family
        
    @property
    def implementations(self):
        """Does game implement another? (e.g. Pandemic: Legacy re-implements Pandemic)"""
        link = self.item['link']
        link_names = [ii['@type'] for ii in link]
        names_index = [i for i, x in enumerate(link_names) if x == 'boardgameimplementation']
        implements = [link[ii]['@value'] for ii in names_index]
        
        return implements
        
    @property
    def designers(self):
        """List of game's designers"""
        link = self.item['link']
        link_names = [ii['@type'] for ii in link]
        names_index = [i for i, x in enumerate(link_names) if x == 'boardgamedesigner']
        designers = [link[ii]['@value'] for ii in names_index]
        
        return designers
        
    @property
    def artists(self):
        """List of game's artists"""
        link = self.item['link']
        link_names = [ii['@type'] for ii in link]
        names_index = [i for i, x in enumerate(link_names) if x == 'boardgameartist']
        artists = [link[ii]['@value'] for ii in names_index]
        
        return artists
        
    @property
    def publishers(self):
        """List of game's Publisher"""
        link = self.item['link']
        try:
            link_names = [ii['@type'] for ii in link]
            names_index = [i for i, x in enumerate(link_names) if x == 'boardgamepublisher']
            publishers = [link[ii]['@value'] for ii in names_index]
        except:
            print(link[0])
            link_names = link[0]['@type']
            names_index = [i for i, x in enumerate(link_names) if x == 'boardgamepublisher']
            publishers = link[0] #['@value']
        return publishers
    
    
    
    
        
    @property
    def rank(self):
        """Dictionary of games various ranks (e.g. Overall, Strategy Games, Family, etc.)"""

        rankings = self.statistics['ratings']['ranks']['rank']
        
        num_rank_names = sum([i == '@name'for i in rankings])
        
        if num_rank_names == 1:
            if type(rankings) is int:
                ranks = {'boardgame':float(rankings['@value'])}
            else:
                ranks = {'boardgame': 0}
        else:
            rank_names = [ii['@name'] for ii in rankings]
            
            ranks = {}
            def _get_ranking(name):
                name_rank = rank_names.index(name)
                if type(rankings[name_rank]['@value']) is int:
                    ranks[name] = int(rankings[name_rank]['@value'])
                else:
                    ranks[name] = 0
            for ii in rank_names:
                _get_ranking(ii)
        
        return ranks
    
    @property
    def ranks_bayes(self):
        """Dictionary of games various bayesian ranks (e.g. Overall, Strategy Games, Family, etc.)"""
        rankings = self.statistics['ratings']['ranks']['rank']
        num_rank_names = sum([i == '@name' for i in rankings])
        
        if num_rank_names == 1:
            ranks = {'boardgame':float(rankings['@bayesaverage'])}
        
        else:
            rank_names = [ii['@name'] for ii in rankings]
            
            ranks = {}
            def _get_bayes_ranking(name):
                name_rank = rank_names.index(name)
                ranks[name] = float(rankings[name_rank]['@bayesaverage'])
            
            for ii in rank_names:
                _get_bayes_ranking(ii)
        
        return ranks
    
    @property
    def users_rated(self):
        """Number of BGG users who've rated game"""
        return int(self.statistics['ratings']['usersrated']['@value'])
        
    @property
    def avg_rating(self):
        """Average BGG user rating (10-point scale)"""
        return float(self.statistics['ratings']['average']['@value'])
        
    @property
    def bayes_avg_rating(self):
        """Average BGG user's bayesian rating (10-point scale)"""
        return float(self.statistics['ratings']['bayesaverage']['@value'])
    
    @property
    def stdev_rating(self):
        """Standard deviation of BGG user rating (10-point scale)"""
        return float(self.statistics['ratings']['stddev']['@value'])
    
    @property
    def median_rating(self):
        """Median BGG user rating (10-point scale)"""
        return float(self.statistics['ratings']['median']['@value'])
        
    @property
    def num_owned(self):
        """Number of BGG users who own this game"""
        return  int(self.statistics['ratings']['owned']['@value'])
        
    @property
    def num_trading(self):
        """Number of copies being traded on BGG marketplace"""
        return int(self.statistics['ratings']['trading']['@value'])
        
    @property
    def num_wanting(self):
        """Number of BGG users who want to buy this game"""
        return int(self.statistics['ratings']['wanting']['@value'])
        
    @property
    def num_wishing(self):
        """Number of BGG users who wish they had this game"""
        return int(self.statistics['ratings']['wishing']['@value'])
        
    @property
    def num_comments(self):
        """Number of comments about the game"""
        return int(self.statistics['ratings']['numcomments']['@value'])
        
    @property
    def num_weights(self):
        """Number of BGG users who have assigned the game a weight (5-point scale)"""
        return int(self.statistics['ratings']['numweights']['@value'])
        
    @property
    def avg_weight(self):
        """Average weight assigned by BGG users (5-point scale)"""
        return float(self.statistics['ratings']['averageweight']['@value'])

## Import necessary packages

First, import all packages necessary for this extraction.

In [2]:
"""Main boardgame script file"""

import pickle
import pandas as pd
import time 
# from urllib2 import urlopen
from urllib.request import urlopen
import requests
import re
from bs4 import BeautifulSoup as BS
import xmltodict
import time
import progressbar
import sys
import numpy as np
import pyodbc
import json
import math
from tqdm import tqdm

In [3]:
total_timer_start = time.time()

## Download pages from the internet (not yet games details, just ID and Name)

This function is used to download the Game ID from BGG search page. This will be helpful to collect information about the game itself later on, based on the IDs.

The Game name is also downloaded, but in the end, it is not used.

In [4]:
def pull_game_names(page):
    """Get all game names and IDs from a page of BGG website.
    
    Description:
        Scrapes Board Game Geek website games in order of rank.
        Returns a dictionary of games titles:BGG ID, given a page number
        Games are listed in increments of 50
        
    inputs:
        page (int): Page number (starts at 1)
        
    returns:
        game_list (dict): {Name:ID}
    """
    url = 'https://boardgamegeek.com/browse/boardgame/page/{}'.format(page)
    bgg_page = urlopen(url)

    my_bytes = bgg_page.read()
    url_text = my_bytes.decode("utf8")
    bgg_page.close()
    url_text = BS(url_text, 'html.parser')
    
    games = url_text.find_all("td", class_="collection_objectname")
    
    def get_game_name(item):
        game_name = item.findNext('a').text
        return(game_name)
        
    def get_game_ID(item):
        game_link_id = str(item.findNext('a'))
        game_link_id = re.search('[0-9]{1,7}', game_link_id).group(0)
        return(int(game_link_id))
    
    game_list = {get_game_name(ii):get_game_ID(ii) for ii in games}
    

    return(game_list)

In [5]:
def list_to_boardgame_class(num_pages):

    """Return list of Boardgame Objects
    
    Description:
        Calls pull_game_names() and boardgame_info(), passing information to 
        Boardgame() class, and returns a list of all class objects.
     
    Input:
        num_pages (int): Number of pages of games (100 per page)
    
    Returns:
        all_games (list): List of each game as a Boardgame object
    
    """
    ## Get boardgame names and IDs for the top 1000 games (10 pages)
    game_list = [pull_game_names(ii) for ii in progressbar.log_progress(range(1, num_pages + 1),'Game List')]
    game_IDs = [x for y in list([ii.values() for ii in game_list]) for x in y]
    ## Connect IDs with BGG API, get game information

    return(game_IDs) #(game_list, game_IDs)

## Procedure Start

We start here by going througth the pages and collecting the BoardGame ID. Then, we append that to a list with all games IDs

### Feedback
Sometimes scripts run for a long time without stopping, specialy with URL request, since those rely on the connection itself. Or even for long loops.
In this case, both things happen, so I added a ProgressBar as a way to know if the script is really running correctly. 

The user feedback function is called **progressbar.log_progress(list, string)**

In [ ]:
start = time.time()

num_pages = 750

if __name__ == "__main__":
    ## Get boardgame names and IDs for the top xxx games (100 * num_pages)
    all_games_IDs = list_to_boardgame_class(num_pages)
    print("Scraping complete")

end = time.time()
print('Elapsed time: ', time.strftime("%H:%M:%S", time.gmtime(end-start)))

In [7]:
len(all_games_IDs)

74972

## Building URL with 500 games to increase download performance

In [8]:
all_games_IDs_backup = all_games_IDs.copy()

In [9]:
ids_already_processed = []
list_of_urls = []
number_of_games_on_search = 450
search_len = math.ceil(len(all_games_IDs)/number_of_games_on_search) #math.ceil will round up to the next integer
if search_len < 1:
    search_len = 1
print('Number of searches:', search_len)

Number of searches: 167


In [10]:
for i in range(int(search_len)):
    url = 'https://boardgamegeek.com/xmlapi2/thing?id='
    for ID in all_games_IDs[:number_of_games_on_search]:
            url = url + str(ID) + ','
            ids_already_processed.append(ID)
            all_games_IDs.remove(ID)
    url= url[:-1] + '&stats=1'
    list_of_urls.append(url)

In [11]:
print('Number of XMLs is', len(list_of_urls), ',with a total of ',len(ids_already_processed),'games')

Number of XMLs is 167 ,with a total of  74972 games


## Download game details

If game IDs collected and URLs group and ready, we can finally download the XMLs from BGG Website

### First try
Sometimes the page returns an error for a few requests. Since this takes a long time to finish, I divided this in two parts

1) First, download pages from the web and store in a list
    a) While doing that, check for errors in the download and store on another list for a second retry

2) With everything local, parse the XMLL. This will avoid having to download everything again in search for errors

### Checkpoint

In [12]:
save_list_of_urls = list_of_urls.copy()

In [13]:
def download_bg_details(list_of_urls_to_download):
    attempts = 0
    for url in progressbar.log_progress(list_of_urls_to_download, 'Downloading Pages (Multiple Games Details)'):
            page = requests.get(url)
            if page.status_code == requests.codes.ok:
                successful_requests.append(page)
            else:
                bad_url_request.append(url)
    if len(bad_url_request) != 0 and attempts <=10:
        download_bg_details(bad_url_request)
        attempts += 1
    return successful_requests

In [14]:
first_half = int(round(len(list_of_urls)/2,0))
second_half = int((len(list_of_urls) - first_half)*-1)

In [15]:
start = time.time()
bad_url_request = []
successful_requests = []

downloaded_xmls = download_bg_details(list_of_urls[:first_half])
downloaded_xmls = download_bg_details(list_of_urls[:second_half])

end = time.time()
print('Elapsed time: ', time.strftime("%H:%M:%S", time.gmtime(end-start)))

Elapsed time:  00:17:45


In [16]:
dup = []
unique = []
for item in downloaded_xmls:
    if item not in unique:
        unique.append(item)
    else:
        dup.append(item)
len(dup)

0

In [17]:
backup_of_downloads = downloaded_xmls.copy

## Breaking down the XMLs received into a list with BoardGame objects

In [18]:
start = time.time()
flatted_list = []

for item in progressbar.log_progress(downloaded_xmls, 'Pages on XML'):
    page_xml = xmltodict.parse(item.content)
    for each_game in page_xml['items']['item']:
        temp = BoardGame(each_game)
        flatted_list.append(temp)
end = time.time()
print('Elapsed time: ', time.strftime("%H:%M:%S", time.gmtime(end-start)))

Elapsed time:  00:01:28


## Build DataFrame

Finaly we build the DataFrame of all games. This process is taking very long to complete. Improvements may be necessary.

In [21]:
lst = flatted_list
all_games = lst[:]
bg_df = pd.DataFrame()

bg_df = pd.DataFrame()
bg_df['Name'] = [i.name for i in progressbar.log_progress(flatted_list, 'Scanning Name')]
bg_df['ID'] = [i.id for i in progressbar.log_progress(flatted_list, 'Scanning ID')] 
try:
    bg_df['Publisher'] = [i.publishers for i in progressbar.log_progress(flatted_list, 'Scanning Publishers')]
except:
    bg_df['Publisher'] = 'Not Found'
    
bg_df['Minimum Players'] = [i.min_players for i in progressbar.log_progress(flatted_list, 'Scanning Min Players')]
bg_df['Max Players'] = [i.max_players for i in progressbar.log_progress(flatted_list, 'Scanning Max Players')]
bg_df['Num_Ratings'] = [i.users_rated for i in progressbar.log_progress(flatted_list, 'Scanning Num Ratings')]
bg_df['Avg_Rating'] = [i.avg_rating for i in progressbar.log_progress(flatted_list, 'Scanning Ratings')]
bg_df['Bayes_Avg_Rating'] = [i.bayes_avg_rating for i in progressbar.log_progress(flatted_list, 'Scanning Bayes Rating')]
bg_df['Avg_Weight'] = [i.avg_weight for i in progressbar.log_progress(flatted_list, 'Scanning AVG Weight')]
bg_df['StDev_Rating'] = [i.stdev_rating for i in progressbar.log_progress(flatted_list, 'Scanning StDev Rating')]
bg_df['Num_Owned'] = [i.num_owned for i in progressbar.log_progress(flatted_list, 'Scanning Num Owned')]
bg_df['Num_Comments'] = [i.num_comments for i in progressbar.log_progress(flatted_list, 'Scanning Num Comments')]
bg_df['Num_Trading'] = [i.num_trading for i in progressbar.log_progress(flatted_list, 'Scanning Num Tradings')]
bg_df['Num_Wanting'] = [i.num_wanting for i in progressbar.log_progress(flatted_list, 'Scanning Num Waiting')]
bg_df['Num_Wishing'] = [i.num_wishing for i in progressbar.log_progress(flatted_list, 'Scanning Num Wishing')]
bg_df['Num_Weights'] = [i.num_weights for i in progressbar.log_progress(flatted_list, 'Scanning Num weights')]
bg_df['Year'] = [i.year_published for i in progressbar.log_progress(flatted_list, 'Scanning Year Published')]

## Rating
bg_df['Num_Ratings'] = [i.users_rated for i in progressbar.log_progress(all_games, 'Scanning Num Ratings')]
bg_df['Avg_Rating'] = [i.avg_rating for i in progressbar.log_progress(all_games, 'Scanning Ratings')]
bg_df['Bayes_Avg_Rating'] = [i.bayes_avg_rating for i in progressbar.log_progress(all_games, 'Scanning Bayes Rating')]
bg_df['Avg_Weight'] = [i.avg_weight for i in progressbar.log_progress(all_games, 'Scanning AVG Weight')]
bg_df['StDev_Rating'] = [i.stdev_rating for i in progressbar.log_progress(all_games, 'Scanning StDev Rating')]

## Players
try:
    bg_df['Pub_Min_Age'] = [i.min_age for i in progressbar.log_progress(all_games, 'Publisher Min Age')]
except:
    bg_df['Pub_Min_Age'] = 0


BGG_Min_Age = []
for i in all_games:
    try:
        BGG_Min_Age.append(i.suggested_player_age)
    except:
        BGG_Min_Age.append(np.nan)

bg_df['BGG_Min_Age'] = BGG_Min_Age

try:
    bg_df['Pub_Min_Players'] = [i.min_players for i in progressbar.log_progress(all_games, 'Publisher Min Players')]
except:
    bg_df['Pub_Min_Players'] = 0

try:
    bg_df['Pub_Max_Players'] = [i.max_players for i in progressbar.log_progress(all_games, 'Publisher Max Players')]
except:
    bg_df['Pub_Max_Players'] = 0

BGG_Num_Players = []
for i in all_games:
    try:
        BGG_Num_Players.append(i.suggested_players)
    except:
        BGG_Num_Players.append(np.nan)

bg_df['BGG_Num_Players'] = BGG_Num_Players
try:
    bg_df['Play_Time'] = [i.playing_time for i in progressbar.log_progress(all_games, 'Scanning Play Time')]
except:
    bg_df['Play_Time'] = 0

## Ownership & Community Engagement
bg_df['Num_Owned'] = [i.num_owned for i in progressbar.log_progress(all_games, 'Scanning Num Owned')]
bg_df['Num_Comments'] = [i.num_comments for i in progressbar.log_progress(all_games, 'Scanning Num Comments')]
bg_df['Num_Trading'] = [i.num_trading for i in progressbar.log_progress(all_games, 'Scanning Num Tradings')]
bg_df['Num_Wanting'] = [i.num_wanting for i in progressbar.log_progress(all_games, 'Scanning Num Waiting')]
bg_df['Num_Wishing'] = [i.num_wishing for i in progressbar.log_progress(all_games, 'Scanning Num Wishing')]
bg_df['Num_Weights'] = [i.num_weights for i in progressbar.log_progress(all_games, 'Scanning Num weights')]
try:
    bg_df['Categories'] = [i.categories for i in progressbar.log_progress(all_games, 'Scanning Categories')]
except:
     bg_df['Categories'] = 0
try:
    bg_df['Mechanics'] = [i.mechanics for i in progressbar.log_progress(all_games, 'Scanning Mechanics')]
except:
    bg_df['Mechanics'] = 0
### Ranks
rank_categories = []
for i in progressbar.log_progress(all_games, 'Rank categories'):
    rank_categories.append(list(i.rank.keys()))

unique_rank_categories = list(set([i for j in progressbar.log_progress(rank_categories, 'Rank categories') for i in j]))
rank_columns = ['Name', 'ID'] + unique_rank_categories
major_rank_df = pd.DataFrame(columns = rank_columns)

for game in progressbar.log_progress(all_games, 'Rank'):
    rank_data = game.rank
    rank_data['Name'] = game.name
    rank_data['ID'] = game.id
    minor_rank_df = pd.DataFrame(rank_data, index=[0], columns = rank_columns)
    major_rank_df = major_rank_df.append(minor_rank_df)

In [22]:
bg_df.head()

,Name,ID,Publisher,Minimum Players,Max Players,Num_Ratings,Avg_Rating,Bayes_Avg_Rating,Avg_Weight,StDev_Rating,...,Num_Weights,Year,Pub_Min_Age,BGG_Min_Age,Pub_Min_Players,Pub_Max_Players,BGG_Num_Players,Play_Time,Categories,Mechanics
0,Gloomhaven,174430,Not Found,1,4,28863,8.86868,8.59398,3.7997,1.59463,...,1218,2017,12,14.0,1,4,3,120,0,0
1,Pandemic Legacy: Season 1,161936,Not Found,2,4,33379,8.62922,8.47506,2.8312,1.60196,...,936,2015,13,12.0,2,4,4,60,0,0
2,Terraforming Mars,167791,Not Found,1,5,44725,8.41172,8.25463,3.2260,1.36877,...,1708,2016,12,12.0,1,5,3,120,0,0
3,Through the Ages: A New Story of Civilization,182028,Not Found,2,4,17377,8.50946,8.24578,4.3830,1.49249,...,859,2015,14,14.0,2,4,3,120,0,0
4,Twilight Struggle,12333,Not Found,2,2,35885,8.31890,8.16854,3.5720,1.57927,...,3313,2005,13,14.0,2,2,2,180,0,0


In [23]:
bg_df_backup = bg_df.copy()

In [24]:
bg_df = bg_df.merge(major_rank_df, on = ['Name', 'ID'])

In [ ]:
bg_df = bg_df.drop_duplicates(subset="ID", keep='first', inplace=False)

In [28]:
bg_df.shape

(153288, 41)

## Remove duplicates

In [29]:
bg_df['BGG Rank'] = [i+1 for i in range(0,bg_df.shape[0])]

In [31]:
bg_df.head(1)

,Name,ID,Publisher,Minimum Players,Max Players,Num_Ratings,Avg_Rating,Bayes_Avg_Rating,Avg_Weight,StDev_Rating,...,thematic,boardgameaccessory,wargames,rpgitem,commodore64,abstracts,amiga,arcade,strategygames,BGG Rank
0,Gloomhaven,174430,Not Found,1,4,28863,8.86868,8.59398,3.7997,1.59463,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1


## Add Today to Dataframe

In [32]:
bg_df['Downloaded Date'] = time.strftime("%Y-%m-%d", time.gmtime(time.time()))

## Saving DataFrame to File and Database

In [43]:
bg_df.to_csv('export.csv')

In [35]:
total_timer_end = time.time()
print('Total elapsed time: ', time.strftime("%H:%M:%S", time.gmtime(total_timer_end-total_timer_start)))

Total elapsed time:  03:05:06


## Cleaning up

#### Check Point

In [47]:
df = bg_df.copy()
df.shape

(153288, 43)

In [49]:
removeColumns = ['childrensgames', 'videogame', 'familygames', 'partygames', 'atarist', 'cgs', 'thematic', \
                 'boardgameaccessory', 'wargames', 'rpgitem', 'commodore64', 'abstracts', 'amiga', 'arcade', 'strategygames']

In [55]:
df = df.drop(columns=removeColumns)

KeyError: "['childrensgames' 'videogame' 'familygames' 'partygames' 'atarist' 'cgs'\n 'thematic' 'boardgameaccessory' 'wargames' 'rpgitem' 'commodore64'\n 'abstracts' 'amiga' 'arcade' 'strategygames'] not found in axis"

In [59]:
df = df.fillna(0)

In [83]:
df = df.drop_duplicates(subset="ID", keep='first')

In [84]:
df.to_csv('BGG Export.csv')

In [85]:
df.head()

,Name,ID,Publisher,Minimum Players,Max Players,Num_Ratings,Avg_Rating,Bayes_Avg_Rating,Avg_Weight,StDev_Rating,...,BGG_Min_Age,Pub_Min_Players,Pub_Max_Players,BGG_Num_Players,Play_Time,Categories,Mechanics,boardgame,BGG Rank,Downloaded Date
0,Gloomhaven,174430,Not Found,1,4,28863,8.86868,8.59398,3.7997,1.59463,...,14.0,1,4,3,120,0,0,0,1,2019-10-29
4,Pandemic Legacy: Season 1,161936,Not Found,2,4,33379,8.62922,8.47506,2.8312,1.60196,...,12.0,2,4,4,60,0,0,0,5,2019-10-29
8,Terraforming Mars,167791,Not Found,1,5,44725,8.41172,8.25463,3.2260,1.36877,...,12.0,1,5,3,120,0,0,0,9,2019-10-29
12,Through the Ages: A New Story of Civilization,182028,Not Found,2,4,17377,8.50946,8.24578,4.3830,1.49249,...,14.0,2,4,3,120,0,0,0,13,2019-10-29
16,Twilight Struggle,12333,Not Found,2,2,35885,8.31890,8.16854,3.5720,1.57927,...,14.0,2,2,2,180,0,0,0,17,2019-10-29


In [87]:
bg_df_backup.to_csv('BGG Export.csv')